<a href="https://colab.research.google.com/github/KaveenAmarasekara/Intellihack_RedHawk_Task01/blob/main/notebooks/intellihack_task01_rain_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Weather Forecasting for Smart Agriculture
This project aims to predict whether it will rain or not based on historical weather data. The dataset includes features like temperature, humidity, and wind speed.